### Self Consistency Prompting

[Self-consistency Improves Chain of Thought Reasoning in Language Models](https://arxiv.org/pdf/2203.11171.pdf)

### Loading Libraries

In [1]:
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.2/390.2 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.6 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.5
    Uninstalling openai-1.54.5:
      Successfully uninstalled openai-1.54.5


In [2]:
# Import the os module to interact with the operating system
import os

# Import the ChatOpenAI class from the langchain_openai package
# This class is used to interact with OpenAI's chat models
from langchain_openai import ChatOpenAI

### Instantiating the OpenAI model

In [3]:
# prompt: mount drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Set the OPENAI_API_KEY environment variable by reading the API key from a file named 'key.txt' located in the parent directory
os.environ["OPENAI_API_KEY"] = open('/content/drive/MyDrive/GenAI classes/Gen AI tobeshared/Demo Notebooks/key.txt','r').read()

In [5]:
# Initialize a ChatOpenAI instance with a temperature of 0.0
# Setting the temperature to 0.0 makes the model's output more deterministic,
# meaning it will prioritize the most confident prediction over others.
llm = ChatOpenAI(temperature = 0.0)

### Defining the Prompt Templates

In [6]:
# Define a template for prompts that includes both system and user messages
# This template is used to structure the input to the chat model, ensuring that both system messages (e.g., instructions or context) and user messages are included in the conversation.
prompt_template = """
{system_message}
{user_message}"""

In [7]:
# Define a system message that instructs the assistant on its role and behavior
# This message sets the context for the assistant to answer queries on financial information without repeating the question and to focus solely on the user's query.
system_message = """
You are an assistant tasked to answer queries on financial information.
Do not repeat the question. Only answer the question presented by the user.
"""

In [8]:
# Define a template for generating answers based on a given context and question
# This template instructs the model to use the provided context to generate a specified number of distinct answers to a given question.
# The answers should be presented in numbered bullet points, focusing solely on the answers without additional context or explanation.

answers_template = """
Context:
{context}
===
Using the context above generate {num_answers} distinct answers to the following question:
Question:
{question}.

Arrange your answers in numbered bullet points.
Present only the answers in bullet points.
"""

In [9]:
# Define a context string that provides an overview of Tesla's financial performance
tesla_annual_report_context ="""
In 2022, we recognized total revenues of $81.46 billion, respectively, representing an increase of $27.64 billion, compared to the prior year.
We continue to ramp production, build new manufacturing capacity and expand our operations to enable increased deliveries and deployments of our
products and further revenue growth.
"""

In [10]:
factual_question = "What was the increase in annual revenue in 2022 compared to 2021?"

In [11]:
# Construct a prompt for the chat model by formatting the prompt template with the system message and a user message
# The user message is generated by formatting the answers template with the Tesla annual report context, a factual question, and specifying the number of answers to generate
# This results in a structured prompt that instructs the model to use the provided context to generate answers to a specific question, with the answers formatted as numbered
# bullet points
answers_prompt = prompt_template.format(
    system_message=system_message,
    user_message=answers_template.format(
        context=tesla_annual_report_context,
        question=factual_question,
        num_answers=3
    )
)

In [12]:
# printing the prompt
print(answers_prompt)



You are an assistant tasked to answer queries on financial information.
Do not repeat the question. Only answer the question presented by the user.


Context:

In 2022, we recognized total revenues of $81.46 billion, respectively, representing an increase of $27.64 billion, compared to the prior year.
We continue to ramp production, build new manufacturing capacity and expand our operations to enable increased deliveries and deployments of our
products and further revenue growth.

===
Using the context above generate 3 distinct answers to the following question:
Question:
What was the increase in annual revenue in 2022 compared to 2021?.

Arrange your answers in numbered bullet points.
Present only the answers in bullet points.



### Generating the Answers

In [13]:
# generating the answers
print(llm.predict(answers_prompt))

<ipython-input-13-4cc353cd8b11>:2: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(llm.predict(answers_prompt))



- The increase in annual revenue in 2022 compared to 2021 was $27.64 billion.
- Total revenues in 2022 were $81.46 billion, representing an increase of $27.64 billion from the prior year.
- The increase in revenue from 2021 to 2022 was $27.64 billion.


In [14]:
# storing the above answers in a variable for further prompting
factual_answers = llm.predict(answers_prompt)

### Defining Self Consistency Prompt Template

In [15]:
# Define a template for evaluating the consistency of generated answers
# This template is designed to present a set number of answers to a specific question,
# instructing the evaluator to identify the most frequently occurring answer among the provided options.
# The final answer should be presented in a clear and concise format, focusing solely on the most common solution.

consistency_template = """
Here are {num_answers} answers to the question mentioned below:
Question:
{question}
Answers:
{answers}

Observe the answers mentioned above and choose the answer that occurs most.
Present only the most frequent solution in the following format.
Final Answer:
"""

In [16]:
# Construct a prompt for evaluating the consistency of generated answers by formatting the prompt template with the system message and a user message
# The user message is generated by formatting the consistency template with the number of answers, the factual question, and the factual answers provided
# This results in a structured prompt that instructs the model to present a set number of answers to a specific question and then choose the most frequent answer among them
consistency_prompt = prompt_template.format(
    system_message=system_message,
    user_message=consistency_template.format(
        num_answers=3,
        question=factual_question,
        answers=factual_answers
    )
)

In [17]:
# printing the consistency prompt
print(consistency_prompt)



You are an assistant tasked to answer queries on financial information.
Do not repeat the question. Only answer the question presented by the user.


Here are 3 answers to the question mentioned below:
Question:
What was the increase in annual revenue in 2022 compared to 2021?
Answers:

- The increase in annual revenue in 2022 compared to 2021 was $27.64 billion.
- Total revenues in 2022 were $81.46 billion, representing an increase of $27.64 billion from the prior year.
- The increase in revenue from 2021 to 2022 was $27.64 billion.

Observe the answers mentioned above and choose the answer that occurs most.
Present only the most frequent solution in the following format.
Final Answer:



### Generating Final Answer

In [18]:
# generating the final answer
llm.predict(consistency_prompt)

'The increase in annual revenue in 2022 compared to 2021 was $27.64 billion.'